In [2]:
pip install tensorflow



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /usr/local/Cellar/jupyterlab/4.0.6/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
